# Predicting Failure in Tanzanian Water Pumps


In [139]:
# Imports
import pandas as pd
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import numpy as np

# Pandas options to make outputs more readable
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

In [140]:
# Create pandas dataframes from csv files
train = pd.merge(pd.read_csv('https://raw.githubusercontent.com/WillHK/DS-Unit-2-Classification-1/master/module1-logistic-regression/train_features.csv')
                ,pd.read_csv('https://raw.githubusercontent.com/WillHK/DS-Unit-2-Classification-1/master/module1-logistic-regression/train_labels.csv'))
test = pd.read_csv('https://raw.githubusercontent.com/WillHK/DS-Unit-2-Classification-1/master/module1-logistic-regression/test_features.csv')
sample_submission = pd.read_csv('https://raw.githubusercontent.com/WillHK/DS-Unit-2-Classification-1/master/module1-logistic-regression/sample_submission.csv')

In [141]:
# Create Training and Validation Sets from the train dataframe

# Stratify keeps the proportions of our target approximately the same in
# the traina and val sets

train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                             stratify=train['status_group'])

# Cleaning Data and Feature Extraction
Now that we have our data in DataFrames and split appropriately we can start looking at it and deciding how we need to modify it.

In [142]:
# Lets look at the train dataframe and see what our data looks like
train.describe(include='all')

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
count,47520.00,47520.00,47520,44647,47520.00,44629,47520.00,47520.00,47520,47520.00,47520,47241,47520,47520.00,47520.00,47520,47520,47520.00,44881,47520,44448,25081,45065,47520.00,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520,47520
unique,nan,nan,349,1683,nan,1902,nan,nan,30772,nan,9,17166,21,nan,nan,125,2077,nan,2,1,12,2542,2,nan,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6,3
top,nan,nan,2011-03-15,Government Of Tanzania,nan,DWE,nan,nan,none,nan,Lake Victoria,Madukani,Iringa,nan,nan,Njombe,Igosi,nan,True,GeoData Consultants Ltd,VWC,K,True,nan,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
freq,nan,nan,469,7312,nan,13939,nan,nan,2879,nan,8199,424,4264,nan,nan,2015,238,nan,40902,47520,29394,554,31165,nan,21548,21548,21548,32374,41981,20319,20319,40701,40701,26596,26596,13677,13677,36562,22878,27775,25807
mean,37133.10,312.22,NaN,NaN,671.52,NaN,34.07,-5.70,NaN,0.49,NaN,NaN,NaN,15.28,5.63,NaN,NaN,180.89,NaN,NaN,NaN,NaN,NaN,1305.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,21457.10,2419.04,NaN,NaN,693.84,NaN,6.58,2.95,NaN,13.03,NaN,NaN,NaN,17.51,9.61,NaN,NaN,478.06,NaN,NaN,NaN,NaN,NaN,950.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.00,0.00,NaN,NaN,-90.00,NaN,0.00,-11.65,NaN,0.00,NaN,NaN,NaN,1.00,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,18514.75,0.00,NaN,NaN,0.00,NaN,33.09,-8.54,NaN,0.00,NaN,NaN,NaN,5.00,2.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37094.50,0.00,NaN,NaN,373.00,NaN,34.91,-5.02,NaN,0.00,NaN,NaN,NaN,12.00,3.00,NaN,NaN,30.00,NaN,NaN,NaN,NaN,NaN,1986.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,55696.25,25.00,NaN,NaN,1322.00,NaN,37.18,-3.32,NaN,0.00,NaN,NaN,NaN,17.00,5.00,NaN,NaN,215.00,NaN,NaN,NaN,NaN,NaN,2004.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## What's the difference between some of these duplicate features?

In [143]:
# payment and payment_type

print('payment\n')
print(train['payment'].value_counts())
print('\npayment_type\n')
print(train['payment_type'].value_counts())

payment

never pay                20319
pay per bucket            7163
pay monthly               6638
unknown                   6492
pay when scheme fails     3134
pay annually              2936
other                      838
Name: payment, dtype: int64

payment_type

never pay     20319
per bucket     7163
monthly        6638
unknown        6492
on failure     3134
annually       2936
other           838
Name: payment_type, dtype: int64


In [144]:
#water_quality and quality_group

print('water_quality\n')
print(train['water_quality'].value_counts())
print('\nquality_group\n')
print(train['quality_group'].value_counts())

water_quality

soft                  40701
salty                  3873
unknown                1487
milky                   634
coloured                391
salty abandoned         254
fluoride                163
fluoride abandoned       17
Name: water_quality, dtype: int64

quality_group

good        40701
salty        4127
unknown      1487
milky         634
colored       391
fluoride      180
Name: quality_group, dtype: int64


# What features don't matter?
To start with id is a unique identifier for each row which will do nothing but add noise to any model we try to create, gotta go.

Two columns are duplicates which we can safely drop, payment_type and quantity_type

There are many columns that look to be the same data just with finer grain, since we'll be using a RandomForestClassifier we can safely keep the more accurate features and drop the rest

In [145]:
train = train.drop(columns=['id', 'extraction_type_group', 'extraction_type_class',
                    'payment_type', 'quality_group', 'quantity_group',
                    'source_type', 'source_class', 'waterpoint_type_group', 'management_group', 'recorded_by'])
val = val.drop(columns=['id', 'extraction_type_group', 'extraction_type_class',
                    'payment_type', 'quality_group', 'quantity_group',
                    'source_type', 'source_class', 'waterpoint_type_group', 'management_group', 'recorded_by'])
test = test.drop(columns=['id', 'extraction_type_group', 'extraction_type_class',
                    'payment_type', 'quality_group', 'quantity_group',
                    'source_type', 'source_class', 'waterpoint_type_group', 'management_group', 'recorded_by'])

In [146]:
train.describe(include='all')

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,management,payment,water_quality,quantity,source,waterpoint_type,status_group
count,47520.00,47520,44647,47520.00,44629,47520.00,47520.00,47520,47520.00,47520,47241,47520,47520.00,47520.00,47520,47520,47520.00,44881,44448,25081,45065,47520.00,47520,47520,47520,47520,47520,47520,47520,47520
unique,nan,349,1683,nan,1902,nan,nan,30772,nan,9,17166,21,nan,nan,125,2077,nan,2,12,2542,2,nan,18,12,7,8,5,10,7,3
top,nan,2011-03-15,Government Of Tanzania,nan,DWE,nan,nan,none,nan,Lake Victoria,Madukani,Iringa,nan,nan,Njombe,Igosi,nan,True,VWC,K,True,nan,gravity,vwc,never pay,soft,enough,spring,communal standpipe,functional
freq,nan,469,7312,nan,13939,nan,nan,2879,nan,8199,424,4264,nan,nan,2015,238,nan,40902,29394,554,31165,nan,21548,32374,20319,40701,26596,13677,22878,25807
mean,312.22,NaN,NaN,671.52,NaN,34.07,-5.70,NaN,0.49,NaN,NaN,NaN,15.28,5.63,NaN,NaN,180.89,NaN,NaN,NaN,NaN,1305.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2419.04,NaN,NaN,693.84,NaN,6.58,2.95,NaN,13.03,NaN,NaN,NaN,17.51,9.61,NaN,NaN,478.06,NaN,NaN,NaN,NaN,950.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.00,NaN,NaN,-90.00,NaN,0.00,-11.65,NaN,0.00,NaN,NaN,NaN,1.00,0.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.00,NaN,NaN,0.00,NaN,33.09,-8.54,NaN,0.00,NaN,NaN,NaN,5.00,2.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.00,NaN,NaN,373.00,NaN,34.91,-5.02,NaN,0.00,NaN,NaN,NaN,12.00,3.00,NaN,NaN,30.00,NaN,NaN,NaN,NaN,1986.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,25.00,NaN,NaN,1322.00,NaN,37.18,-3.32,NaN,0.00,NaN,NaN,NaN,17.00,5.00,NaN,NaN,215.00,NaN,NaN,NaN,NaN,2004.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Any nan values to deal with?

In [148]:
# For many numeric entries 0 or a float close enough to 0 is equivalent to nan
def clean_zeros(df):
    df_c = df.copy()
    df_c['latitude'] = df_c['latitude'].replace(-2e-08, np.nan)
    zero_cols = ['construction_year', 'longitude', 'latitude', 'gps_height',
                 'population']
    for col in zero_cols:
      df_c[col] = df_c[col].replace(0, np.nan)
    return df_c

train = clean_zeros(train)
val = clean_zeros(val)
test = clean_zeros(test)

train.isnull().sum()

amount_tsh               0
date_recorded            0
funder                2873
gps_height           16244
installer             2891
longitude             1454
latitude              1454
wpt_name                 0
num_private              0
basin                    0
subvillage             279
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population           17014
public_meeting        2639
scheme_management     3072
scheme_name          22439
permit                2455
construction_year    16461
extraction_type          0
management               0
payment                  0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
status_group             0
dtype: int64

## Yes, many. 

For the categoricals we will replace them with 'MISSING' and for the numeric entires we will take the mean of the region they're in.

In [151]:
categoricals = train.select_dtypes(exclude='number').columns.tolist()
for col in categoricals:
  train[col] = train[col].fillna('MISSING')
  val[col] = val[col].fillna('MISSING')
  if col != 'status_group':
    test[col] = test[col].fillna('MISSING')

## This has a syntax error I don't understand
## so I'm using SimpleImputer below in the pipeline

# numerical_features = train.select_dtypes(include='number').columns.tolist()

# def regional_mean(df, features):
#   df_c = df.copy()
#   for i in range(len(df_c)):
#     for feature in features:
#       df_c = df_c[feature].fillna(value=df_c[df_c['region'] == df_c.iloc[i]['region']][feature].mean())

#   return df_c


# train = regional_mean(train, numerical_features)
# val = regional_mean(val, numerical_features)
  
# train.isnull().sum()

## Convert dates to a more tree friendly format

By converting date_recorded to a datetime and extracting the year, month and day as new features we can safely drop the 'date_recorded column and have 3 new features that should be better for the Random Forest model.



In [152]:
train['date_recorded'] = pd.to_datetime(train['date_recorded'], infer_datetime_format=True)
train['year_recorded'] = train['date_recorded'].dt.year
train['month_recorded'] = train['date_recorded'].dt.month
train['day_recorded'] = train['date_recorded'].dt.day
train = train.drop(columns='date_recorded')

val['date_recorded'] = pd.to_datetime(val['date_recorded'], infer_datetime_format=True)
val['year_recorded'] = val['date_recorded'].dt.year
val['month_recorded'] = val['date_recorded'].dt.month
val['day_recorded'] = val['date_recorded'].dt.day
val = val.drop(columns='date_recorded')

test['date_recorded'] = pd.to_datetime(test['date_recorded'], infer_datetime_format=True)
test['year_recorded'] = test['date_recorded'].dt.year
test['month_recorded'] = test['date_recorded'].dt.month
test['day_recorded'] = test['date_recorded'].dt.day
test = test.drop(columns='date_recorded')

# Create X and y dataframes 

In [153]:
target = 'status_group'

X_train = train.drop(columns=[target])
y_train = train[target]
X_val = val.drop(columns=[target])
y_val = val[target]
X_test = test

# Create simple pipeline for our model

This pipeline will simply take the X and y dataframes as parameters in the fit method, ordinally encode the categorical features and run an Random Forest Classifier on it.

In [154]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=1000, max_depth=40, n_jobs=-1)
)

## Fit the model and check the score

By using a pipeline we can easily fit our model and test as we experiment with our dataset.

In [157]:
pipeline.fit(X_train, y_train)
print(pipeline.score(X_val, y_val))

0.8095959595959596


In [158]:
y_pred = pipeline.predict(X_test)
submission = pd.read_csv('https://raw.githubusercontent.com/WillHK/DS-Unit-2-Classification-1/master/module1-logistic-regression/sample_submission.csv')
submission['status_group'] = y_pred
submission.to_csv('submission-08.csv', index=False)